# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import os
os.chdir('..')

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf

In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

Reading configuration from ../src/config/default.conf
INFO:tensorflow:Restoring parameters from ..\models/gib21_model/gib21-1000000
INFO:tensorflow:Restoring parameters from ..\models/gib21_info_model/gib21_info-500000
INFO:tensorflow:Restoring parameters from ..\models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/lefty_model/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\models/dummy_model/dummy-920000
INFO:tensorflow:Restoring parameters from ..\models/righty_model/righty-1000000
INFO:tensorflow:Restoring parameters from ..\models/decl_model/decl-1000000
Reading configuration from ../src/config/default.conf


In [3]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'S'
vuln = [True, True]  # fist element is NS, second element is EW

hands = [
    'AJ87632.J96.753.',
    'K9.Q8542.T6.AJ74',
    'QT4.A.KJ94.KQ986',
    '5.KT73.AQ82.T532'
]

auction = ['1N', 'PASS', '4H', 'PASS', '4S', 'PASS', 'PASS', 'PASS']

play = ['C2', 'D3', 'CA', 'C6', 'D6', 'DJ', 'DQ', 'D5', 'DA', 'D7', 'DT', 'D4', 'D8', 'H6', 'H2', 'D9', 'SQ', 'S5', 'S2', 'SK', 'H4', 'HA', 'H7', 'H9', 'S4', 'C3', 'SA', 'S9', 'S3', 'C4', 'ST', 'H3', 'CK', 'C5', 'HJ', 'C7', 'C8', 'CT', 'S6', 'CJ', 'S7', 'H8', 'C9', 'D2', 'S8', 'H5', 'CQ', 'HT', 'SJ', 'HQ', 'DK', 'HK']

In [5]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler)

In [ ]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

card_by_card.analyze()

analyzing the bidding
1N ?
PASS .
4H .
PASS .
4S .
PASS .
PASS .
PASS .
analyzing the play
C2
D3 ??
CA .
C6 .
D6 .
DJ .
DQ .
D5 .
DA .
D7 .
DT .
D4 .
D8 .
H6 ?
H2 .
D9 .
SQ .
S5 .
S2 .
SK .
H4 .
HA .
H7 .
H9 .
S4 .
C3 .
SA .
S9 .
S3 .
C4 .
ST .
H3 .
CK .
C5 .
HJ .
C7 .
C8 .
CT .
S6 .
CJ .
S7 .
H8 .
C9 .
D2 .
S8 .
H5 .
CQ .
HT .


In [ ]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

{'bid': '1N',
 'candidates': [{'call': '1C', 'insta_score': 0.9971140623092651}],
 'samples': ['J8xx.Qxx.xxx.Jxx AK9x.JT8x.Qx.xxx QTx.A.KJ9x.KQ98x xx.K9xxx.AT8x.AT',
  'KJxxx.K9x.8xx.xx Ax.Qxxxx.Txx.Txx QTx.A.KJ9x.KQ98x 98x.JT8x.AQx.AJx',
  'KJ9xx.8xx.Q.JTxx A8x.T9xx.Txx.Axx QTx.A.KJ9x.KQ98x xx.KQJxx.A8xxx.x',
  'xxx.QJx.8xxx.ATx J9.T9xxxx.QTx.xx QTx.A.KJ9x.KQ98x AK8xx.K8x.Ax.Jxx',
  '8x.Qxxxx.Axxx.Ax AJxxx.KT8.T.JTxx QTx.A.KJ9x.KQ98x K9x.J9xx.Q8xx.xx',
  'Kx.T8xxx.Ax.ATxx 9xxx.KQ9x.8xx.xx QTx.A.KJ9x.KQ98x AJ8x.Jxx.QTxx.Jx',
  'AJ9x.QTx.Txx.Jxx xx.8xxx.AQxx.Axx QTx.A.KJ9x.KQ98x K8xx.KJ9xx.8x.Tx',
  'J8xx.8xxx.Ax.Axx AKx.QJT9x.8xx.Jx QTx.A.KJ9x.KQ98x 9xx.Kxx.QTxx.Txx',
  'xx.9xxx.AQxx.JTx J98x.KT8xx.8x.xx QTx.A.KJ9x.KQ98x AKxx.QJx.Txx.Axx',
  'Jxx.KQx.Ax.AJTxx K98xx.Txx.Q8xx.x QTx.A.KJ9x.KQ98x Ax.J98xxx.Txx.xx']}

the engine very confidently opens `1C` and doesn't even consider `1N`

In [ ]:
# what about the opening lead? let's see...

card_by_card.cards['C2'].to_dict()

{'card': 'C2',
 'candidates': [{'card': 'Cx',
   'insta_score': 0.6596599221229553,
   'expected_tricks': 10.647597105337653,
   'p_make_contract': 0.8120009197560654},
  {'card': 'Hx',
   'insta_score': 0.2709367275238037,
   'expected_tricks': 10.672620729739785,
   'p_make_contract': 0.815934700474523}],
 'samples': ['AK8xxx.Jx.JT.J98 T9xx.Q9xx.xx.Axx QJ.A8x.K9xxx.KQx',
  'KQJ9xx.98x.9x.Kx T8.AJxx.Jxxx.J8x Axxx.Qx.KTx.AQ9x',
  'AQ9xxxx.Jx.x.8xx KJ8.8xxx.K9x.Q9x Tx.AQ9.JTxxx.AKJ',
  'KQJ98xx.x.Tx.A98 xx.Q98x.K9xx.Jxx ATx.AJxx.Jxx.KQx',
  'AQJT8xxx.x.x.98x Kx.Q8xx.KJT9x.xx 9x.AJ9x.xxx.AKQJ',
  'QT98xxxx.9.K9.Qx x.QJ8x.JTxxx.KJx AKJ.Axxx.xx.A98x',
  'AKT8xxxx.x.K9.8x 9x.QJ98x.Txx.QJx QJ.Axx.Jxxx.AK9x',
  'QJTxxx.9x.Tx.AQJ K9xx.QJ8x.9x.9xx A8.Axx.KJxxx.K8x',
  'Q98xxx.QJ98x..A8 Txx.x.KJTxx.QJ9x AKJ.Axx.9xxx.Kxx',
  'KJT8xx.Q.JTx.Q8x AQ9x.98xxx.xxx.9 xx.AJx.K9x.AKJxx',
  'AJ9xxx.J8x.x.K8x KTx.Q9xx.Txx.J9x Q8x.Ax.KJ9xx.AQx',
  'AKJ9xx.QJ.Txx.Qx T8x.98x.J9x.J9xx Qxx.Axxx.Kxx.AK8',
  'AQJxx

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [ ]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

{'card': 'D3',
 'candidates': [{'card': 'S2',
   'insta_score': 0.4216527682246551,
   'expected_tricks': 11.125,
   'expected_score': 622.7},
  {'card': 'S6',
   'insta_score': 0.4216527682246551,
   'expected_tricks': 11.125,
   'expected_score': 622.7},
  {'card': 'S7',
   'insta_score': 0.4216527682246551,
   'expected_tricks': 11.125,
   'expected_score': 622.7},
  {'card': 'S3',
   'insta_score': 0.4216527682246551,
   'expected_tricks': 11.125,
   'expected_score': 622.7},
  {'card': 'S8',
   'insta_score': 0.038019460358720523,
   'expected_tricks': 11.125,
   'expected_score': 622.7},
  {'card': 'SJ',
   'insta_score': 0.0,
   'expected_tricks': 10.98,
   'expected_score': 608.0},
  {'card': 'D7',
   'insta_score': 0.4681495904723088,
   'expected_tricks': 10.395,
   'expected_score': 538.7},
  {'card': 'D3',
   'insta_score': 0.4681495904723088,
   'expected_tricks': 10.395,
   'expected_score': 538.7},
  {'card': 'D5',
   'insta_score': 0.4681495904723088,
   'expected_trick

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)

## Analyzing a board played on BBO

In [ ]:
# copy-paste from the hand records (in lin format)

lin = 'pn|You,~~M7228oka,~~M72302cm,~~M72316sq|st||md|1S4TKHJD68QC679TKA,S35H479TQKD24TAC8,S2789H3AD379JKC35,|rh||ah|Board 3|sv|e|mb|1C|an|Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points|mb|2H|an|Aggressive weak jump overcall -- 6+ !H; 4-10 HCP |mb|d|an|Negative double -- 4+ !S; 7+ HCP; 8+ total points |mb|4H|an|The Law: 10 trump -> game support -- 4+ total points |mb|4S|an|3+ !C; 4+ !S; 16-21 HCP; 17-22 total points|mb|p|mb|p|mb|p|pg||pc|DA|pc|D3|pc|D5|pc|D6|pg||pc|C8|pc|C3|pc|CJ|pc|CA|pg||pc|S4|pc|S5|pc|S8|pc|SJ|pg||pc|H5|pc|HJ|pc|HQ|pc|HA|pg||pc|S2|pc|SA|pc|ST|pc|S3|pg||pc|H2|pc|SK|pc|H4|pc|H3|pg||pc|D8|pc|D2|pc|DJ|pc|S6|pg||pc|SQ|pc|C6|pc|H7|pc|S7|pg||pc|H8|pc|C7|pc|HK|pc|S9|pg||pc|C5|pc|C2|pc|CT|pc|HT|pg||pc|CK|pc|H9|pc|D7|pc|C4|pg||pc|DQ|pc|D4|pc|DK|pc|H6|pg||pc|D9|pc|CQ|pc|C9|pc|DT|pg||'

In [ ]:
display_lin(lin)

In [ ]:
board = parse_lin(lin)

In [ ]:
card_by_card = CardByCard(*board, models)

In [ ]:
card_by_card.analyze()

analyzing the bidding
1C .
2H .
X .
4H .
4S .
PASS .
PASS .
PASS .
analyzing the play
DA
D3 .
D5 .
D6 .
C8 .
C3 .
CJ .
CA .
S4 ?
S5 .
S8 .
SJ .
H5 ?
HJ .
HQ .
HA .
S2 .
SA .
ST .
S3 .
H2 .
SK ?
H4 .
H3 .
D8 .
D2 .
DJ .
S6 .
SQ .
C6 .
H7 .
S7 .
H8 .
C7 .
HK .
S9 .
C5 .
C2 .
CT .
HT .
CK .
H9 .
D7 .
C4 .
DQ .
D4 .
DK .
H6 .


the engine agrees with the bidding, but didn't like something in the cardplay.

playing `S4` from hand is the first mistake. apparently this play drops almost half a trick on average.

In [ ]:
card_by_card.cards['S4'].to_dict()

{'card': 'S4',
 'candidates': [{'card': 'D8',
   'insta_score': 0.1471998191220598,
   'expected_tricks': 7.815789473684211,
   'expected_score': -59.21052631578947},
  {'card': 'DQ',
   'insta_score': 0.04370090231684083,
   'expected_tricks': 7.815789473684211,
   'expected_score': -59.21052631578947},
  {'card': 'HJ',
   'insta_score': 0.11742140918900698,
   'expected_tricks': 7.473684210526316,
   'expected_score': -76.3157894736842},
  {'card': 'S4',
   'insta_score': 0.047128788500056425,
   'expected_tricks': 7.2894736842105265,
   'expected_score': -85.52631578947368},
  {'card': 'CK',
   'insta_score': 0.18763520523410357,
   'expected_tricks': 7.2368421052631575,
   'expected_score': -88.15789473684211},
  {'card': 'ST',
   'insta_score': 0.014114008965710806,
   'expected_tricks': 7.184210526315789,
   'expected_score': -90.78947368421052},
  {'card': 'SK',
   'insta_score': 0.1934124744049229,
   'expected_tricks': 7.078947368421052,
   'expected_score': -96.05263157894737

the opening lead of `DA` is interesting. the engine prefers the `HK` and it's the only card it considers.

In [ ]:
card_by_card.cards['DA'].to_dict()

{'card': 'DA',
 'candidates': [{'card': 'HK',
   'insta_score': 0.8880357146263123,
   'expected_tricks': 10.450010813546761,
   'p_make_contract': 0.7548369560424516}],
 'samples': ['J98x.x.KJxx.Qxxx Txx.A8xxx.Q8.Kxx AKQx.J.9xx.AJT9x',
  'J9xxx.A.Q8x.Jxxx KT.J8xxx.K9xx.Tx AQ8x.x.Jx.AKQ9xx',
  'Q9xxxx..KQJx.xxx KT.J8xxxx.xx.9xx AJ8.A.98x.AKQJTx',
  'QT8xx.A.J9xx.KTx 9x.J8xx.Qxx.Axxx AKJx.xx.K8.QJ9xx',
  'AK98.8x.Q98x.QTx Jxx.AJxx.Jxx.J9x QTxx.x.Kx.AKxxxx',
  'AJ98x.A8.J98x.xx Txx.xxxx.xx.KT9x KQx.J.KQx.AQJxxx',
  'KQ8x.8x.K98x.QJx ATxx.Jxxx.QJ.T9x J9x.A.xxx.AKxxxx',
  'JTxx.8x.K9x.KTxx KQx.Jxxx.QJ8x.9x A98x.A.xx.AQJxxx',
  'KQxxx.A8.Q9xx.J9 8.Jxxx.J8xx.Axxx AJT9x.x.K.KQTxxx',
  'KQJ8.xx.9x.Kxxxx xxx.AJ8xx.QJxx.x AT9x..K8x.AQJT9x',
  'AQ9xx.x.98xxx.Kx Jx.J8xxx.Jx.9xxx KT8x.A.KQ.AQJTxx',
  'AQ9xx.x.9xxx.Jxx JTx.8xxx.J8x.KQx K8x.AJ.KQ.AT9xxx',
  'A8xx.x.Q98x.QJxx QJT9.Jxxx.xxx.xx Kxx.A8.KJ.AKT9xx',
  'Txxx.Ax.KJ98.ATx K8x.J8xx.Qxxx.Qx AQJ9.x.x.KJ9xxxx',
  'AJ9xx.x.KQ8x.J9x QTx.A8xx.J9x.Tx